In [1]:
import pandas as pd
import numpy as np
import random
import baselines as b
from sklearn.metrics import accuracy_score,f1_score, recall_score, precision_score, average_precision_score, balanced_accuracy_score

In [2]:
#path='./data/SAT_Predictions1/'
path='./data/MRA_Files/'
#path='./data/SAT_findings/'



In [3]:
def normal_abnormal(path,i):
    
    data_with_tags = np.load('./data/Data_with_tags.npy',allow_pickle=True)
    
    #MRA
    orig_data=pd.read_json(path+'gts_'+str(i)+'.json')
    
    idx=['CXR'+str(i) for i in list(orig_data.columns)]
    tags_sat = [data_with_tags[()][r] for r in idx]
    
    
    
    
    #for SAT
    
#     orig_data=pd.read_csv(path+'score'+str(i)+'.csv')
    
#     print(orig_data.shape)
    
#     tags_sat = [data_with_tags[()][r] for r in orig_data['REPORT_ID']]
    
    
    is_normal_sat = np.array(['normal' in t for t in tags_sat])
    
    return is_normal_sat

In [4]:


def accuracy_comp(data,test_data,is_normal):
    
#     is_normal=data['No Finding']==1
    
    
    
    data1=data.loc[is_normal,:]
    test_data1=test_data.loc[is_normal,:]
    
    return accuracy_score(data1.to_numpy().flatten(),test_data1.to_numpy().flatten())

sample=[2,3,4,5]

bleu1,bleu2,bleu3,bleu4,M, R, C=[],[],[],[],[],[],[]

accuracy,precision,recall_mac,recall_mic=[],[],[],[]

classes=['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Lesion', 'Lung Opacity', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices']

for i in sample:
    
    print('sample:', i)
    gt=pd.read_csv(path+'gts_labelled'+str(i)+'.csv')
    test=pd.read_csv(path+'res_labelled'+str(i)+'.csv')
    
    
    
    is_normal=normal_abnormal(path,i)
    #gt=pd.read_csv(path+'score_ref_labelled'+str(i)+'.csv')
    
    #test=pd.read_csv(path+'score_labelled'+str(i)+'.csv')
    #gt=pd.read_csv(path+'score_ref_labelled'+str(i)+'.csv')
    
#     print(gt.shape)
    
    #test=pd.read_csv(path+'score_labelled'+str(i)+'.csv')
    
#     print(test.shape)
    
#     baselines=pd.read_csv('./data/baselines_latest.csv')
    
#     #print(baselines.iloc[:,0])
#     test=baselines.iloc[2,:]

#     test=pd.DataFrame(test).T

#     test=pd.concat([test]*gt.shape[0], ignore_index=True)


    
    gt=gt.fillna(0.0)


    gt=gt.replace(-1,0)

    test=test.fillna(0.0)


    test=test.replace(-1,0)
    
    data=gt.iloc[:,1:]

    test_data=test.iloc[:,1:]

    #test_data=test_data.sample(frac=1)
#     n=test_data.shape[0]
#     r=pd.DataFrame(data.iloc[report_id,:]).T

    pred = test_data#r * 1.0
    
    
    #### accuracy=======================================

    accuracy.append(accuracy_comp(data,test_data,is_normal))

    #print(accuracy)
#     comp = test_data.values * pred.values + (1 - test_data.values) * (1 - pred.values)

#     #print(comp.shape)
    
#     accu = np.mean(comp, axis=1)
    
    
    
#     accuracy.append(np.mean(accu))

#     print('accuracy:', np.mean(accu))
    
    
    ######## Precision ===============================
    precision.append(precision_score(data.to_numpy().flatten(),test_data.to_numpy().flatten()))

    #print(precision)
    
    
    ######## Recall Micro ===============================
    recall_mic.append(recall_score(data.to_numpy().flatten(),test_data.to_numpy().flatten()))

    #print(recall_mic)
    
     ######## Recall Macro ===============================
        
    ######################### Class wise computation

    
    recl_class_wise=[]

    for c in classes:
        
        recl_class_wise.append(recall_score(data[c],test_data[c]))

    
    recall_mac.append(np.mean(recl_class_wise))

    

#     print(recall_mac)
    
    
    #is_normal=gt['No Finding']==0
    
#     print(is_normal)
#     bleus,meteor, rouge, ciders=b.all_scores(gt.iloc[:,0],test.iloc[:,0])
    bleus,meteor, rouge, ciders=b.all_scores(np.array(gt.iloc[:,0])[is_normal], test.iloc[2,0])
    
    bleu1.append(bleus[0])
    bleu2.append(bleus[1])
    bleu3.append(bleus[2])
    bleu4.append(bleus[3])
    M.append(meteor)
    R.append(rouge)
    C.append(ciders)

print(np.mean(bleu1),np.mean(bleu2),np.mean(bleu3),np.mean(bleu4),np.mean(M),np.mean(R),np.mean(C))
print(np.std(bleu1),np.std(bleu2),np.std(bleu3),np.std(bleu4),np.std(M),np.std(R),np.std(C))

print(np.mean(accuracy),np.mean(precision),np.mean(recall_mac),np.mean(recall_mic))

sample: 2
{'testlen': 6887, 'reflen': 2680, 'guess': [6887, 6790, 6693, 6596], 'correct': [1767, 815, 423, 224]}
ratio: 2.569776119402026
sample: 3
{'testlen': 4500, 'reflen': 2472, 'guess': [4500, 4410, 4320, 4230], 'correct': [1427, 633, 313, 159]}
ratio: 1.8203883495138267
sample: 4
{'testlen': 4232, 'reflen': 2591, 'guess': [4232, 4140, 4048, 3956], 'correct': [1480, 714, 376, 158]}
ratio: 1.633346198378374
sample: 5
{'testlen': 3713, 'reflen': 2295, 'guess': [3713, 3634, 3555, 3476], 'correct': [1284, 566, 281, 114]}
ratio: 1.6178649237465716
0.3173024797551267 0.2166256548180299 0.15334543689144864 0.10667264043151131 0.23463388575190483 0.307521755179372 0.03204741571332161
0.037255492382254986 0.026366059025805848 0.019343133404326824 0.011426359518856203 0.009643347389578295 0.011273611887375735 0.020852050708955898
0.9123213588873678 0.26372443977591037 0.07283077778821134 0.21637696903042228
